In [11]:
from ai2thor.controller import Controller
from tasks import ServeCoffee, ServeEgg
import scene_info as si
from pddl2scene import PlanFileHandler
from scene2pddl import SceneHandler

controller = Controller(
    agentMode="default",
    visibilityDistance=1.5,
    scene="FloorPlan3",

    # step sizes
    gridSize=0.25,
    snapToGrid=True,
    rotateStepDegrees=90,

    # image modalities
    renderDepthImage=False,
    renderInstanceSegmentation=False,

    # camera properties
    width=320,
    height=180,
    fieldOfView=100
)

In [12]:
tasks = [ServeCoffee(),
         ServeEgg()]

In [13]:
last_scene = si.Scene(controller)
pddl_filename = SceneHandler(last_scene, tasks).ToPDDL()

print(pddl_filename)

File pddl/problem_857604.pddl written successfully
pddl/problem_857604.pddl


In [ ]:

scene_model = sp.scene2pddl(last_scene)

task = ServeCoffee()
scene_model.subtasks = [task]

print(scene_model.predicates)

In [3]:
event = controller.step(
    action="DropHandObject",
    objectId="Pot|-01.58|+01.31|-01.58",
    forceAction=False
)
info = sp.SceneInfo(controller, event)   
print([obj.metadata for obj in info.objects if obj.metadata["id"]=="CoffeeMachine|+01.13|+01.31|-01.43"]) 

AttributeError: module 'scene2pddl' has no attribute 'SceneInfo'

In [ ]:
#print([obj.metadata["id"] for obj in info.objects])

# if obj.metadata["type"] == "coffee"])

In [ ]:
controller.step(action="PickupObject", objectId="Box|-00.47|+01.04|-00.71")